## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,9/12/16,Citibike to Work,1.15,NaN,11.0,0.18,6.27,Citibike
1,8/31/20,To Marimekko,2.60,NaN,21.0,0.35,7.43,Citibike
2,5/26/20,Down Up LWS,13.35,1.0,30.0,1.50,8.90,Specialized
3,10/7/16,Citibike to Work,1.22,NaN,12.0,0.20,6.10,Citibike
4,5/5/21,Dn UWS Up ES,21.25,2.0,31.0,2.52,8.44,Specialized
...,...,...,...,...,...,...,...,...
364,6/21/16,Citibike to Work,1.12,NaN,10.0,0.17,6.72,Citibike
365,8/24/20,Dn WS Up ES,17.12,2.0,5.0,2.08,8.22,Specialized
366,8/22/20,Wards Island x 2,17.58,2.0,12.0,2.20,7.99,Specialized
367,9/4/20,From Astor Wines,3.72,NaN,31.0,0.52,7.20,Citibike


In [5]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,9/12/16,Citibike to Work,1.15,0.0,11.0,0.18,6.27,Citibike
1,8/31/20,To Marimekko,2.60,0.0,21.0,0.35,7.43,Citibike
2,5/26/20,Down Up LWS,13.35,1.0,30.0,1.50,8.90,Specialized
3,10/7/16,Citibike to Work,1.22,0.0,12.0,0.20,6.10,Citibike
4,5/5/21,Dn UWS Up ES,21.25,2.0,31.0,2.52,8.44,Specialized
...,...,...,...,...,...,...,...,...
364,6/21/16,Citibike to Work,1.12,0.0,10.0,0.17,6.72,Citibike
365,8/24/20,Dn WS Up ES,17.12,2.0,5.0,2.08,8.22,Specialized
366,8/22/20,Wards Island x 2,17.58,2.0,12.0,2.20,7.99,Specialized
367,9/4/20,From Astor Wines,3.72,0.0,31.0,0.52,7.20,Citibike


In [6]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,9/12/16,Citibike to Work,1.15,0.0,11.0,0.18,6.27,Citibike
1,1,8/31/20,To Marimekko,2.60,0.0,21.0,0.35,7.43,Citibike
2,2,5/26/20,Down Up LWS,13.35,1.0,30.0,1.50,8.90,Specialized
3,3,10/7/16,Citibike to Work,1.22,0.0,12.0,0.20,6.10,Citibike
4,4,5/5/21,Dn UWS Up ES,21.25,2.0,31.0,2.52,8.44,Specialized
...,...,...,...,...,...,...,...,...,...
364,364,6/21/16,Citibike to Work,1.12,0.0,10.0,0.17,6.72,Citibike
365,365,8/24/20,Dn WS Up ES,17.12,2.0,5.0,2.08,8.22,Specialized
366,366,8/22/20,Wards Island x 2,17.58,2.0,12.0,2.20,7.99,Specialized
367,367,9/4/20,From Astor Wines,3.72,0.0,31.0,0.52,7.20,Citibike


In [7]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [8]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,1.15,0.18,6.27,Citibike
1,2.60,0.35,7.43,Citibike
2,13.35,1.50,8.90,Specialized
3,1.22,0.20,6.10,Citibike
4,21.25,2.52,8.44,Specialized
...,...,...,...,...
364,1.12,0.17,6.72,Citibike
365,17.12,2.08,8.22,Specialized
366,17.58,2.20,7.99,Specialized
367,3.72,0.52,7.20,Citibike


In [9]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [10]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [11]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [12]:
# Checking training data shape
X_train.shape

(276, 3)

In [13]:
# Checking testing data shape
X_test.shape

(93, 3)

In [14]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [15]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [16]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [17]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.967741935483871


In [18]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Citibike
1,Citibike
2,Specialized
3,Citibike
4,Specialized
...,...
88,Citibike
89,Citibike
90,Citibike
91,Specialized


In [19]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
17,Citibike
157,Citibike
78,Specialized
132,Specialized
146,Specialized
...,...
138,Citibike
165,Citibike
339,Citibike
176,Specialized


In [20]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,17,Citibike
1,157,Citibike
2,78,Specialized
3,132,Specialized
4,146,Specialized
...,...,...
88,138,Citibike
89,165,Citibike
90,339,Citibike
91,176,Specialized


In [21]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,17,Citibike,Citibike
1,157,Citibike,Citibike
2,78,Specialized,Specialized
3,132,Specialized,Citibike
4,146,Specialized,Specialized
...,...,...,...
88,138,Citibike,Citibike
89,165,Citibike,Citibike
90,339,Citibike,Citibike
91,176,Specialized,Specialized


In [22]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
3,132,Specialized,Citibike
25,221,Specialized,Citibike
47,304,Citibike,Specialized


In [23]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,132,Specialized,Citibike,5/25/20,Uptown Trio,4.27,0.0,34.0,0.57,7.54,Specialized
1,221,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54,Specialized
2,304,Citibike,Specialized,6/17/16,Work to Village,4.69,0.0,40.0,0.67,7.04,Citibike


In [24]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,132,Specialized,Citibike,5/25/20,Uptown Trio,4.27,0.0,34.0,0.57,7.54
1,221,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54
2,304,Citibike,Specialized,6/17/16,Work to Village,4.69,0.0,40.0,0.67,7.04
